### simple RAG using [VARAG](https://github.com/adithya-s-k/VARAG)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adithya-s-k/VARAG/blob/main/docs/simpleRAG.ipynb)

Requirement to RUN this notebook - CPU or T4(if using OCR and need fast OCR)

In [ ]:
!git clone https://github.com/adithya-s-k/VARAG
%cd VARAG
%pwd

In [ ]:
!apt-get update && apt-get install -y && apt-get install -y poppler-utils

In [ ]:
%pip install -e .

## We will be using Docling for OCR
%pip install docling

In [1]:
from sentence_transformers import SentenceTransformer
from varag.rag import SimpleRAG
from varag.llms import OpenAI
from varag.llms import LiteLLM
from varag.chunking import FixedTokenChunker
import lancedb
import os
from dotenv import load_dotenv

# os.environ["OPENAI_API_KEY"] = "api-key"

load_dotenv()

/home/adithya/miniconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


ModuleNotFoundError: No module named 'varag.chunking'

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# Initialize OpenAI LLM
# llm = LiteLLM(model="gpt-4o-mini" , is_vision_required=True, api_key=OPENAI_API_KEY) 
llm = LiteLLM(model="gpt-3.5-turbo" , is_vision_required=True, api_key=OPENAI_API_KEY) 

In [ ]:
response = llm.query(query="What is you name?" , )
print(response)

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", trust_remote_code=True)
# embedding_model = SentenceTransformer("BAAI/bge-base-en", trust_remote_code=True)
# embedding_model = SentenceTransformer("BAAI/bge-large-en-v1.5", trust_remote_code=True)
# embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5", trust_remote_code=True)

# Initialize shared database
shared_db = lancedb.connect("~/shared_rag_db")

# Initialize TextRAG with shared database
text_rag = SimpleRAG(
    text_embedding_model=embedding_model,
    db=shared_db,
    table_name="textDemo",
)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Initialize OpenAI LLM
llm = LiteLLM(model="gpt-4o-mini" , is_vision_required=True, api_key=OPENAI_API_KEY)

In [ ]:
text_rag.index(
        "./examples/data",
        recursive=False,
        chunking_strategy=FixedTokenChunker(chunk_size=1000),
        metadata={"source": "gradio_upload"},
        overwrite=True,
        verbose=True,
        ocr=True,
    )

In [ ]:
query = "what is colpali ?"
num_results = 5

search_results = text_rag.search(query, k=num_results)

print("This was the retrieved Context")
for i, r in enumerate(search_results):
    print(f"{'==='*50}")
    print(f"\n\nChunk {i+1}:")
    print(f"Text: {r['text']}")
    print(f"Chunk Index: {r['chunk_index']}")
    print(f"Document Name: {r['document_name']}")
    print(f"\n\n{'==='*50}")

In [ ]:
from IPython.display import display, Markdown, Latex

context = "\n".join([r["text"] for r in search_results])
response = llm.query(
    context=context,
    system_prompt="Given the below information answer the questions",
    query=query,
)


display(Markdown(response))

### Run Gradio Demo

In [ ]:
%cd examples
!python textDemo.py --share